In [6]:
import pandas as pd
from fuzzywuzzy import fuzz

In [7]:
gvkeyToCik = pd.read_csv("gvkey_cik.csv")
cusipToCik = pd.read_csv("cusip8_cik.csv")
gvkeyToCusip = pd.read_csv("gvkey_cusip.csv", encoding = "latin1")
mainTable = pd.read_csv("mainTable.csv")

## Cleaning data

In [9]:
gvkeyToCik["CIK Number"] = gvkeyToCik["CIK Number"].apply(lambda x: str(x).zfill(10))
gvkeyToCik.rename(columns = {"Compustat's Global Company Key - GVKEY" : "Gvkey", 
                             "CIK Number" : "CIK",
                             "Company Name in SEC Filings - Header" : "companyName"}, inplace = True)
gvkeyToCik["Gvkey"] = gvkeyToCik["Gvkey"].astype(str)
gvkeyToCik_By_Name = gvkeyToCik.loc[:, ["companyName", "CIK"]]
gvkeyToCik = gvkeyToCik.loc[:, ["Gvkey", "CIK"]]

# Assigning 0 to the row whose GVKEY value is NAN
mainTable.drop([dropName for dropName in mainTable.columns if "Unnamed" in dropName], axis = 1, inplace = True)
mainTable.loc[mainTable["GVKEY"].isna(), "GVKEY"] = 0
mainTable.GVKEY = mainTable.GVKEY.astype(int).astype(str)
mainTable.rename(columns = {"GVKEY" : "Gvkey", "Institution Name " : "companyName"}, inplace = True)

gvkeyToCusip.rename(columns = {"gvkey" : "Gvkey"}, inplace = True)
gvkeyToCusip = gvkeyToCusip.loc[:, ["Gvkey", "cusip8"]]
gvkeyToCusip.dropna(inplace = True)
gvkeyToCusip["Gvkey"] = gvkeyToCusip["Gvkey"].astype(int).astype(str)

cusipToCik.rename(columns = {"SEC CIK Number" : "CIK",
                             "8-Digit CUSIP Number as Appears in SEC Filings of CIK Entity" : "cusip8",
                             "Company Name" : "companyName"},
                 inplace = True)
cusipToCik["CIK"] = cusipToCik["CIK"].astype(int).astype(str).apply(lambda x : x.zfill(10))
cusipToCik["cusip8"] = cusipToCik["cusip8"].astype(str)
cusipToCik_By_Name = cusipToCik.loc[:, ["companyName", "CIK"]]
cusipToCik = cusipToCik.loc[:, ["cusip8", "CIK"]]

## merging files

In [13]:
gvkey_CIK_matching_table = mainTable.merge(gvkeyToCik, how = "left", left_on = "Gvkey", right_on = "Gvkey")
remaining_table = gvkey_CIK_matching_table.loc[gvkey_CIK_matching_table.CIK.isna(), :]
gvkey_CIK_matching_table = gvkey_CIK_matching_table[~gvkey_CIK_matching_table.CIK.isna()]
remaining_with_cusip = remaining_table.merge(gvkeyToCusip, how = "left", left_on = "Gvkey", right_on = "Gvkey")
remaining_with_cusip.drop("CIK", axis = 1, inplace = True)
remaining_with_CIK = remaining_with_cusip.merge(cusipToCik, how = "left", left_on = "cusip8", right_on = "cusip8")

## fuzzy mataching for the unmatched CIK

In [12]:
# mapping from gvkeyToCik_By_Name to mainTable
for index, word in enumerate(remaining_with_CIK["companyName"]):
    temp = [fuzz.partial_ratio(word, matchingword) for matchingword in gvkeyToCik_By_Name["companyName"]]
    if max(temp) > 70:
        Max_Prob_Index = temp.index(max(temp))
        remaining_with_CIK.iloc[index, -1] = gvkeyToCik_By_Name["CIK"][Max_Prob_Index]

        
# mapping from cusipToCik_By_Name to mainTable
for index, word in enumerate(remaining_with_CIK["companyName"]):
    temp = [fuzz.partial_ratio(word, matchingword) for matchingword in cusipToCik_By_Name["companyName"]]
    if max(temp) > 70:
        Max_Prob_Index = temp.index(max(temp))
        remaining_with_CIK.iloc[index, -1] = cusipToCik_By_Name["CIK"][Max_Prob_Index]

## Output files

In [32]:
remaining_with_CIK.drop("cusip8", axis = 1, inplace = True)
finalTable = pd.concat([gvkey_CIK_matching_table, remaining_with_CIK])
finalTable.to_csv("CIK_File.csv", index = False)